In [1]:
import wandb
from kaggle_secrets import UserSecretsClient
wandb_key = UserSecretsClient().get_secret("wannabe")
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ngjabach (ngjabach-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_name = 'facebook/bart-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Using device: cuda


In [3]:
import pandas as pd

class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])

def tokenize_data(train_question, train_context):
    encodings = tokenizer(train_context, truncation=True, padding=True, max_length = 512)
    decodings = tokenizer(train_question, truncation=True, padding=True, max_length = 512)
    dataset_tokenized = TokenDataset(encodings, decodings)
    return dataset_tokenized

df = pd.read_csv("/kaggle/input/qag-wop/QAG_Train_wop.csv")
df.rename(columns = {'question':'question', 'context':'context'}, inplace = True)
df.keys()

train_question, train_context = (list(df['question'])), (list(df['context']))
train_data = tokenize_data(train_question, train_context)

print(train_data)

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    run_name='bart-large-finetuning',
    output_dir='./results',
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=2000,
    logging_dir=None,
    per_device_train_batch_size=1,
    num_train_epochs=8,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=train_data
)

trainer.train()

model.save_pretrained("bart-baseline")

Step,Training Loss
2000,1.231300
4000,0.296900
6000,0.269900
8000,0.211200
10000,0.163100
12000,0.110500
14000,0.090500
16000,0.059700
18000,0.050000
20000,0.039000


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [5]:
dialogue = "Introduce yourself as detailed as possible."

input_ids = tokenizer(dialogue, return_tensors='pt', 
                      max_length=1024, truncation=True).input_ids.to(device)
output = model.generate(input_ids, max_length=1024, early_stopping=False)

summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Summary: {summary}")

Summary: What do you mean by introducing yourself?


In [7]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('bart-baseline/model.safetensors')

/kaggle/working


/kaggle/working/bart-baseline/model.safetensors